# MultinomialNB

## Estimando o melhor modelo do classificador para o dataset anotado tweetSentBR (sobreamostrado), com vetorizador Tfidf

### Importando bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Carregando os dados para treinamento do modelo

In [ ]:
# carregando dataset
tweets_df = pd.read_csv('')

In [ ]:
tweets_df.head()

In [ ]:
tweets_df.info()

In [ ]:
tweets_df.groupby(['sentiment']).count()

Como podemos observar, os dados estão desbalanceados e com algumas linhas apresentando a variável sentimento com o caracter traço '-'.

Para melhor trabalharmos com os dados, foi decidida a remoção dessas linhas.

In [ ]:
# removendo as linhas com a variável sentimento como traço '-'
indexNames = tweets_df[ tweets_df['sentiment'] == '-' ].index #obtendo o valor para ajudar a remover a linha
tweets_df.drop(indexNames , inplace=True) #removendo a linha correspondente ao valor informado

In [ ]:
# tranformando valores para adequar a os classificadores
tweets_df['sentiment'] = tweets_df['sentiment'].replace(['-1', '0', '1'], [3, 6, 9])
tweets_df['sentiment'] = tweets_df['sentiment'].replace([3, 6, 9], [0, 1, 2])

In [ ]:
# Visualizando os dados graficamente
sns.countplot(x=tweets_df.sentiment)
plt.xlabel('class label')
plt.ylabel('number of tweets')
plt.show()

In [ ]:
tweets_df.groupby(['sentiment']).count()

## Efetuando a limpeza dos dados

In [ ]:
# removendo as palavras USERNAME e NUMBER
tweets_df['text_clean'] = tweets_df['text'].str.replace('USERNAME', '').str.replace('NUMBER', '')

In [ ]:
!python -m nltk.downloader stopwords

In [ ]:
# importando a biblioteca de processamento de linguagen natural e a de expreções regulares (regex)
import nltk
import re

# usando as funções stopword e remover os caracteres indesejados
def Preprocessing(instancia):
    instancia = re.sub(r"http\S+", "", instancia).lower() # remove url e tranforma o texto em minusculo
    instancia = re.sub(r'[0-9]+', '', instancia) # remove números
    instancia = re.sub(r'[^\w\s]','', instancia) # remove pontuação
    instancia = re.sub('[!#$%^&*()]', '', instancia) #remove caractéres espaciais
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [ ]:
# remover emojis utilizado regex (re)
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
# aplicando a função de limpeza de dados:
tweets_df['text_clean'] = [Preprocessing(i) for i in tweets_df['text_clean']]
tweets_df['text_clean'] = [remove_emoji(i) for i in tweets_df['text_clean']]
pd.set_option('display.max_colwidth', None) 

tweets_df.head()

# Lematização

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download pt_core_news_sm

In [ ]:
import spacy
nlp = spacy.load('pt_core_news_sm')

In [ ]:
tweets_df['text_lemma'] = tweets_df['text_clean'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
tweets_df['text_lemma'] = [Preprocessing(i) for i in tweets_df['text_lemma']]
tweets_df.head()

## Lematização (outra forma)

In [ ]:
!pip install simplemma

In [ ]:
import simplemma
from simplemma import text_lemmatizer

In [ ]:
# teste
teste_lemma = text_lemmatizer('Sou o intervalo entre o que desejo ser e os outros me fizeram.', lang='pt')
teste_lemma

In [ ]:
tweets_df['text_lemma_2'] = [text_lemmatizer(linha, lang='pt') for linha in tweets_df['text_clean']] #lematizando o texto
tweets_df['text_lemma_2'] = [' '.join(palavra) for palavra in tweets_df['text_lemma_2']] #juntando o texto novamente
tweets_df.head()

## Trabalhando no equilibrio dos dados

### Iniciando o trabalho de balanceamento dos dados

In [ ]:
minoria = tweets_df.loc[tweets_df['sentiment'] == 0]
meio = tweets_df.loc[tweets_df['sentiment'] == 1]
maioria = tweets_df.loc[tweets_df['sentiment'] == 2]

In [ ]:
print('Maioria \n{}\n'.format(maioria['sentiment'].count()))
print('Meio \n{}\n'.format(meio['sentiment'].count()))
print('Minoria \n{}'.format(minoria['sentiment'].count()))

## Igualar a quantidade de tweets entre positivo, negativo e neutro

#### Oversampling (Criando datasets reorganizados para maioria)

In [ ]:
from sklearn.utils import resample

menor_maior = resample(minoria, replace=True, n_samples=len(maioria), random_state=123)
meio_maior = resample(meio, replace=True, n_samples=len(maioria), random_state=123)

In [ ]:
print(menor_maior.count(), meio_maior.count(), maioria.count())

In [ ]:
# juntando os dataframes
tweets_df_equilibrado_maior = pd.concat([menor_maior, meio_maior, maioria])

# resetando o index
tweets_df_equilibrado_maior = tweets_df_equilibrado_maior.reset_index()

# removendo as colunas Unnamed: 0 e index
tweets_df_equilibrado_maior.drop(columns=['index'], inplace=True)

In [ ]:
tweets_df_equilibrado_maior.groupby(['sentiment']).count()

# Machine Learning - Utils

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# Vetorizadores
from sklearn.feature_extraction.text import TfidfVectorizer

# Classificadores
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# Tokenização especial para tweets
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()

# Machine Learning - Construindo modelo com dados oversampling

In [ ]:
X = tweets_df_equilibrado_maior['text_lemma_2']
y = tweets_df_equilibrado_maior['sentiment']

In [ ]:
X.shape, y.shape

### Separando dados para o treino e para o teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
#parameters = {'modelo__alpha': (0.01, 0.1, 0.5, 1.0, 10.0)}
parameters = {'modelo__alpha': [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]}

In [ ]:
modelo = Pipeline(steps=[
             ('vectorizer' , TfidfVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)),
            ('modelo', MultinomialNB())
            ])

In [ ]:
!pip install searchgrid

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(modelo, parameters, refit = 'accuracy', verbose=3)
clf.fit(X_train, y_train).best_score_

In [ ]:
# Imprime colunas de interesse.
df = pd.DataFrame(clf.cv_results_)[['params','rank_test_score','mean_test_score']]

In [ ]:
df.sort_values(by='rank_test_score', inplace=True)
df

In [ ]:
# salvando o relatório consolidado em excel
df.to_excel("repost_multinomialnb.xlsx")